In [ ]:
# ------------------------------------------------

import os

#source /etc/os-release && echo $PRETTY_NAME
with open('/etc/os-release') as fh:
  for line in fh:
    if line and line[0] != '#':
      os.environ[line.split('=')[0]] = line.split('=')[1]
print(f'Running {os.environ["PRETTY_NAME"]}') # on {os.environ["HOST"]} as {os.environ["USER"]}')

# ------------------------------------------------

from IPython.display import display, clear_output
import time, sys
tick = time.time()
# Install mamba without resetting the kernel alla condacolab
!wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
!bash Mambaforge-Linux-x86_64.sh -bfp /usr/local
sys.path.append('/usr/local/lib/python3.10/site-packages')
!mamba config --set auto_update_conda false
!pip -q install rdkit
!pip -q install pdb2pqr
!pip install -q git+https://github.com/matteoferla/DTC-compchem-practical.git
!mamba install -y -c conda-forge -c bioconda vina oddt openbabel
!pip install -q prolif MDAnalysis
!mamba install -y -c hcc adfr-suite
tock = time.time()
clear_output()
print(f'Installation time: {tock - tick}')

# ------------------------------------------------

import warnings
with warnings.catch_warnings():
  warnings.simplefilter("ignore")
  # last year used pyrosetta... the warning is distracting.
  import DTC_compchem_practical as dtc

In [ ]:
import pkg_resources, os, subprocess
from pathlib import Path

prepped_template_filaname = 'template.pdbqt'
prepped_ligand_filaname = 'ligand.pdbqt'
template_filename: str = pkg_resources.resource_filename('DTC_compchem_practical', 'data/mac1-stripped.pdb')
pipe = subprocess.Popen(f'prepare_receptor -r {template_filename} '+
                        f'-o {prepped_template_filaname}',
                           stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
output, error = pipe.communicate()
print(output.decode('utf-8'))
assert pipe.returncode == 0, f"Error: {error.decode('utf-8')}"
assert Path('/content/template.pdbqt').exists(), 'prepare_receptor failed??'

hit_filename: str = pkg_resources.resource_filename('DTC_compchem_practical', 'data/QRU.pdb')
pipe = subprocess.Popen(f'prepare_ligand -l {hit_filename} '+\
                        f'-o {preped_ligand_filaname} -A hydrogens',
                           stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
output, error = pipe.communicate()
print(output.decode('utf-8'))
assert pipe.returncode == 0, f"Error: {error.decode('utf-8')}"
assert Path('/content/hit.pdbqt').exists(), 'prepare_ligand failed??'
print(f'pdbqt files {prepped_template_filaname} and {prepped_ligand_filaname} made')

In [ ]:
# The ligand `QRU` was derived from 6FZ and BHA
# so we will use their centroid for placement
from rdkit import Chem
from rdkit.Chem import AllChem

# 6FZ and BHA hits used as templates for the derivative QRU
template_hits = [Chem.MolFromMolBlock(b) for b in map(dtc.get_data, ('6FZ.mol', 'BHA.mol'))]
center = list(AllChem.ComputeCentroid( Chem.CombineMols(*template_hits).GetConformer()))

# ------------------------------------------------

from rdkit import Chem
from rdkit.Chem import AllChem
from vina import Vina
# score function to use Vina or another, see help(Vina) for more
vina = Vina(sf_name='vina')
vina.set_receptor(prepped_template_filaname)
vina.set_ligand_from_file(prepped_ligand_filaname)
vina.compute_vina_maps(center=center, box_size=[20, 20, 20])
energy_minimized = vina.optimize()
print(f'Score after minimization : {energy_minimized[0]} (kcal/mol)')
vina.write_pose('vina_minimized.pdbqt', overwrite=True)
vina.dock(exhaustiveness=32, n_poses=20)
vina.write_poses('vina_out.pdbqt', n_poses=10, overwrite=True)

In [ ]:
import MDAnalysis as mda
import prolif as plf

u = mda.Universe(protein_file)
protein_mol = plf.Molecule.from_mda(u)
# display (remove `slice(260, 263)` to show all residues)
plf.display_residues(protein_mol, slice(260, 263))